In [ ]:
from chebgreen.greenlearning.model import *
from chebgreen.chebpy2 import Chebfun2, Chebpy2Preferences
import matplotlib.pyplot as plt

# %env TF_MLC_LOGGING=1 
# %env TF_CPP_MIN_LOG_LEVEL=2

In [ ]:
from chebgreen.chebpy2.chebpy.core.algorithms import chebpts2

In [ ]:
model = GreenNN()
dimension = 1
domain = [0,1,0,1]
layerConfig = [25,25]
activation = 'rational'
homogeneousBC = True
# loadPath = 'savedModels/laplace'
loadPath = 'savedModels/laplace'
model.build(dimension, domain, layerConfig, activation, homogeneousBC, loadPath)

In [ ]:
%%time
cheb2prefs = Chebpy2Preferences()
cheb2prefs.prefx.eps = 1e-6
cheb2prefs.prefx.eps = 1e-6
g = Chebfun2(model.evaluateG, domain = domain, prefs = cheb2prefs, simplify = False)

In [ ]:
g

In [ ]:
%%time
u,s,v = g.svd()

In [ ]:
g.plot()

In [ ]:
g

In [ ]:
%%time
u,s,v = g.svd()

In [ ]:
u[:,:10].plot()

In [ ]:
v[:10,:].plot()

In [ ]:
def green(x,s):
    np.random.seed(42)
    g = 0
    g = (x <= s) * (x * (1-s)) + (x > s) * (s * (1-x))
    # g = g + np.random.normal(loc = 0.0, scale = 0.25 * eps * factor, size=x.shape)
    return g

In [ ]:
cheb2prefs = Chebpy2Preferences()
cheb2prefs.prefx.eps = 1e-6
cheb2prefs.prefx.eps = 1e-6

In [ ]:
%%time
h = Chebfun2(green, domain = [0, 1, 0, 1], prefs = cheb2prefs)

In [ ]:
%%time
_, exact, _ = h.svd()

In [ ]:
fig = plt.figure(figsize = (10,8))
plt.loglog(np.linspace(1,s.shape[0],s.shape[0]),s,'b-x',markersize = 4, alpha = 0.5)
plt.loglog(np.linspace(1,exact.shape[0],exact.shape[0]),exact,'ro',markersize = 4, alpha = 0.3)
# plt.xlim([1,100])
plt.legend(['Computed','Exact'])
# fig.savefig(f'svd-laplace-relu', dpi = 200)

In [ ]:
K=100
plt.tight_layout()
plt.subplots_adjust(left = 0.05, wspace=0.3, hspace=0.3)
f, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize = (24,6))
u[:,:K].plot(fig = f, ax = ax1)
ax1.set_xlim([0,1])
ax1.set_xlabel('$s$')

ax2.loglog(np.linspace(1,s.shape[0],s.shape[0]),s,'b-x',markersize = 4, alpha = 0.5)
ax2.loglog(np.linspace(1,exact.shape[0],exact.shape[0]),exact,'ro',markersize = 4, alpha = 0.3)
ax2.set_xlim([1,100])
ax2.legend(['Computed','Exact'])
ax2.set_ylim([1e-7,1])

ax2.set_xlabel('$k$')
v[:K,:].plot(fig = f, ax = ax3)
ax3.set_xlim([0,1])
ax3.set_xlabel('$x$')
# f.savefig(f'chebGreen.png', dpi = fig.dpi)

In [ ]:
def compareGreen1D(interpModel, exactModel, vmin = None, vmax = None):
    domain = interpModel.domain
    xx = np.linspace(domain[2],domain[3],2000)
    yy = np.linspace(domain[0],domain[1],2000)
    x, s = np.meshgrid(xx,yy)
    
    if vmin is None:
        vmin = min(interpModel.min,exactModel.min)
    if vmax is None:
        vmax = max(interpModel.max,exactModel.max)
        
    plt.figure(figsize = (16,6))
    plt.subplot(121)
    
    GInterp = interpModel[x,s]
    
    levels = np.linspace(vmin, vmax, 50, endpoint = True)
    ticks = np.linspace(vmin, vmax, 10, endpoint=True)
    
    surf = plt.contourf(x, s, GInterp, levels = levels, cmap = 'turbo', vmin = vmin, vmax = vmax)
    plt.gca().set_aspect('equal', adjustable='box')
    cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
    cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])
    plt.xlabel('$x$', fontsize = 11)
    plt.ylabel('$s$', rotation='horizontal', labelpad=10, fontsize = 11)
    plt.title("Interpolated", fontsize = 14)
    
    GExact = exactModel[x,s]
    
    plt.subplot(122)
    plt.gca().set_aspect('equal', adjustable='box')
    surf = plt.contourf(x, s, GExact, levels = levels, cmap = 'turbo', vmin = vmin, vmax = vmax)
    cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
    cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])
    plt.xlabel('x', fontsize = 11)
    plt.ylabel('s', rotation='horizontal', labelpad=10, fontsize = 11)
    plt.title("Data", fontsize = 14)

In [ ]:
i = slice(15,20)
fig = plt.figure(figsize = (8,6))
g.cols[:,i].plot()
print(np.max(g.cols[:,i].vscale))
plt.title(f'Columns of the C, in a CDR decomposition of G')
plt.xlim([0,1])
# fig.savefig(f'Laplace-first.png', dpi = fig.dpi)

In [ ]:
%%time
u,s,v = g.svd()

In [ ]:
# i = slice(10,20)
i = slice(0,10)
# i = 16
fig = plt.figure(figsize = (8,6))
u[:,i].plot()
print(np.max(u[:,i].vscale))
# plt.title(f'First {i+1} left singular functions for a Laplacian')
plt.xlim([0,1])
# fig.savefig(f'Laplace-first.png', dpi = fig.dpi)

In [ ]:
plt.figure()
plt.plot(s)

In [ ]:
xx = np.linspace(domain[0],domain[1],2000)
yy = np.linspace(domain[2],domain[3],2000)
x, y = np.meshgrid(xx,yy)

In [ ]:
%%time
Gneural = model.evaluateG(x,y)

fig = plt.figure(figsize = (8,6))
plt.contourf(x,y,Gneural, 50, cmap = 'turbo', vmin = np.min(Gneural), vmax = np.max(Gneural))
plt.colorbar()
# fig.savefig(f'poisson_greenlearning.png', dpi = fig.dpi)

In [ ]:
%%time
xx = np.linspace(domain[2],domain[3],2000)
yy = np.linspace(domain[0],domain[1],2000)
x, y = np.meshgrid(xx,yy)

G = g[x,y]
fig = plt.figure(figsize = (8,7))
vmin, vmax = np.min(G), np.max(G)
levels = np.linspace(vmin, vmax, 50, endpoint = True)
plt.contourf(x,y,G, levels = levels, cmap = 'jet', vmin = vmin, vmax = vmax)

ticks = np.linspace(vmin, vmax, 10, endpoint=True)
cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])

plt.xlabel(f'$x$', fontsize = 14)
plt.ylabel(f'$s$', rotation='horizontal', labelpad=11, fontsize = 14)
fig.savefig(f'poissonchebfun.png', dpi = fig.dpi)

In [ ]:
vmin, vmax = 0,0.25
Gplot = g

xx = np.linspace(domain[0],domain[1],2000)
yy = np.linspace(domain[2],domain[3],2000)
x, y = np.meshgrid(xx,yy)
Gplot = Gplot[x,y]

fig = plt.figure(figsize = (13,10), frameon=False)
plt.axis('off')
plt.gca().set_aspect('equal', adjustable='box')
levels = np.linspace(vmin, vmax, 50, endpoint = True)
plt.contourf(x,y,Gplot, levels = levels, cmap = 'jet', vmin = vmin, vmax = vmax)
# ticks = np.linspace(vmin, vmax, 6, endpoint=True)
# cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)

fig.savefig('contlaplace-chebGreen.png', dpi = fig.dpi, bbox_inches='tight', pad_inches=0)

In [ ]:
vmin, vmax = 0,0.25
Gplot = green

xx = np.linspace(domain[0],domain[1],2000)
yy = np.linspace(domain[2],domain[3],2000)
x, y = np.meshgrid(xx,yy)
Gplot = Gplot(x,y)

fig = plt.figure(figsize = (13,10), frameon=False)
plt.axis('off')
plt.gca().set_aspect('equal', adjustable='box')
levels = np.linspace(vmin, vmax, 50, endpoint = True)
plt.contourf(x,y,Gplot, levels = levels, cmap = 'jet', vmin = vmin, vmax = vmax)
# ticks = np.linspace(vmin, vmax, 6, endpoint=True)
# cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)

fig.savefig('contlaplace-true.png', dpi = fig.dpi, bbox_inches='tight', pad_inches=0)

In [ ]:
N = 10
x = chebpts2(N)
fig = plt.figure(figsize = (10,3))
plt.plot(x, x * 0, 'k.-')
for xx in x:
    if xx == -1:
        plt.annotate(f'${{{xx:.2f}}}$', xy=(xx, 0), xytext=(-30, 10), textcoords='offset points')
    elif xx == 1:
        plt.annotate(f'${{{xx:.2f}}}$', xy=(xx, 0), xytext=(0, 10), textcoords='offset points')
    else:
        plt.annotate(f'${{{xx:.2f}}}$', xy=(xx, 0), xytext=(-10, 10), textcoords='offset points')
plt.annotate(f'$x$', xy=(0, 0), xytext=(0, -15), textcoords='offset points')
plt.axis('off')

fig.savefig(f'chebpts.png', dpi = 200, bbox_inches='tight', pad_inches=0)

In [ ]:
def normL2(G, meshweights):
    meshweights = meshweights.reshape((-1,1))
    return np.sqrt(np.sum(meshweights * (G*G) * meshweights.T))

def errorL2(G_emp, G, meshweights):
    return normL2(G_emp-G, meshweights)/normL2(G, meshweights)

In [ ]:
def get_weights(x):
    """Trapezoidal weights for trapezoidal rule integration."""
    diff = np.diff(x, axis = 0)
    weights = np.zeros(x.shape, dtype = np.float64)
    weights[1:-1] = diff[1:] + diff[:-1]
    weights[0] = diff[0]
    weights[-1] = diff[-1]
    weights = weights / 2
    return weights

In [ ]:
x = np.linspace(0,1,11)

In [ ]:
get_weights(x)

In [ ]:
xx = np.linspace(domain[2],domain[3],2000)
yy = np.linspace(domain[0],domain[1],2000)
x, y = np.meshgrid(xx,yy)

In [ ]:
G = g[x,y]

In [ ]:
Gexact = green(x,y)

In [ ]:
meshweights = get_weights(xx)

In [ ]:
errorL2(G,Gexact,meshweights)